In [1]:
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

# Set the theme once
pio.templates.default = "plotly_white"

from util import multi_select_counts, get_subplot_coords, df_prop, df_multiselect_prop
import mappings

In [ ]:
df_pre = pl.read_csv("OriginalData/Preprogram_Survey.csv")
df_pre = df_pre.with_columns(
    pl.col("What is your current education status?").map_elements(lambda x: "True" if ("Currently" in x) else "False", return_dtype=pl.String).alias("College Student"),
    pl.col("What is your gender identity?").map_elements(lambda x: "Female" if ("Female" == x) else "Not Female", return_dtype=pl.String).alias("Gender"),
    pl.lit(1).alias("Count") 
).filter( pl.col("College Student") == "True" ) 

df_post = pl.read_csv("./OriginalData/PostOrientation_Survey.csv")
N_post = df_post.shape[0]

df_post = df_post.with_columns(
    pl.col("What is your current education status?").map_elements(lambda x: "True" if ("Currently" in x) else "False", return_dtype=pl.String).alias("College Student"),
    pl.col("What is your gender identity?").map_elements(lambda x: "Female" if ("Female " == x) else "Not Female", return_dtype=pl.String).alias("Gender"),
    pl.lit(1).alias("Count") 
).filter( pl.col("College Student") == "True" ) 

In [18]:
related_columns = {
    'How confident are you now in managing your emotions and staying calm when things get stressful at work or in life?': 'I can manage challenges without giving up',
    'After the Communication Styles workshop, *how well do you understand your own communication style?*': 'Communicating professionally (verbal and written)',
    'What is your current clarity level about your possible career paths?': 'Do you have a *career plan or clear next steps?*',
    'How confident are you in your ability *to manage your money* this summer?': 'Financial knowledge and money management', 
    'After the personal branding session, how confident are you in *how you show up professionally*?': 'I know how to present myself professionally in different settings',
}

In [22]:
standardization_list = [
    "Emotional Regulation",
    "Communication Style",
    "Career Clarity",
    "Financial Management",
    "Professional Presentation"
]

In [23]:
standardization_map = {
    'How confident are you now in managing your emotions and staying calm when things get stressful at work or in life?': {
        'Not confident at all ': 1,
        'A little confident ': 2,
        'Somewhat confident ': 3,
        'Very confident ': 4,
        'Extremely confident ': 5
    },
    'After the Communication Styles workshop, *how well do you understand your own communication style?*': {
        'I still don’t really know it ': 1,
        'I have a general idea ': 2,
        'I understand it clearly ': 4,
        'I know it and I’m already working to improve it ': 5
    },
    'What is your current clarity level about your possible career paths?': {
        'I still feel unsure ': 1,
        'I have a little more clarity now ': 2,
        'I have solid ideas ': 4,
        'I feel very clear about my options and direction ': 5
    },
    'How confident are you in your ability *to manage your money* this summer?': {
        'Not confident at all ': 1,
        'Somewhat confident ': 3,
        'Very confident ': 4,
        'Extremely confident ': 5
    },
    'After the personal branding session, how confident are you in *how you show up professionally*?': {
        'Not confident yet ': 1,
        'A little more confident ': 2,
        'Somewhat confident ': 3,
        'Very confident ': 4,
        'Extremely confident in how I present myself ': 5
    },
    'I can manage challenges without giving up': {
        'Strongly disagree': 1,
        'Disagree': 2,
        'Neutral ': 3,
        'Agree': 4,
        'Strongly agree': 5,
        None: None  # Keeping None as is, or you could map it to 0 or 3 depending on your preference
    },
    'Communicating professionally (verbal and written)': {
        'Not confident': 1,
        'Slightly ': 2,
        'Somewhat': 3,
        'Very': 4,
        'Extremely': 5
    },
    'Do you have a *career plan or clear next steps?*': {
        'I’m still exploring and need guidance': 1,
        'I have a general idea but need help organizing it': 3,
        'Yes, I have a clear plan I’m working on': 5,
        None: None
    },
    'Financial knowledge and money management': {
        'Not confident': 1,
        'Slightly ': 2,
        'Somewhat': 3,
        'Very': 4,
        'Extremely': 5
    },
    'I know how to present myself professionally in different settings': {
        'Strongly disagree': 1,
        'Neutral ': 3,
        'Agree': 4,
        'Strongly agree': 5,
        None: None
    }
}

In [ ]:
df_pre.select(list(related_columns.values())).

I can manage challenges without giving up,Communicating professionally (verbal and written),Do you have a *career plan or clear next steps?*,Financial knowledge and money management,I know how to present myself professionally in different settings
str,str,str,str,str
"""Agree""","""Very""","""I have a general idea but need…","""Very""","""Agree"""
"""Agree""","""Slightly ""","""Yes, I have a clear plan I’m w…","""Slightly ""","""Neutral """
"""Strongly agree""","""Extremely""","""I’m still exploring and need g…","""Very""","""Strongly agree"""
"""Strongly agree""","""Somewhat""","""I have a general idea but need…","""Extremely""","""Strongly agree"""
"""Neutral ""","""Somewhat""","""I’m still exploring and need g…","""Slightly ""","""Agree"""
…,…,…,…,…
"""Strongly disagree""","""Not confident""","""I’m still exploring and need g…","""Slightly ""","""Strongly disagree"""
"""Neutral ""","""Somewhat""","""I have a general idea but need…","""Somewhat""","""Agree"""
"""Neutral ""","""Somewhat""","""I have a general idea but need…","""Somewhat""","""Neutral """
